In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.tree
import seaborn as sns
from scipy import stats

## 导入资管月底明细数据

In [2]:
# 每月导入最新的月底明细
dat1701=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180131.csv')
dat1702=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180228.csv')
dat1703=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180331.csv')
dat1704=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180430.csv')
dat1705=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180531.csv')
dat1706=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180630.csv')
dat1707=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180731.csv')
dat1708=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180831.csv')
dat1709=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20180930.csv')
dat1710=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20181031.csv')
dat1711=pd.read_csv('E:\\pydata\\jhjj_PQR\\collection_jhjj_20181130.csv')

## 合成一张大表dat_bal，只包含“在还”和“逾期”的合同

In [3]:
frames=[dat1701,dat1702,dat1703,dat1704,dat1705,dat1706,dat1707,dat1708,dat1709,dat1710,dat1711]  ## concat合并
dat_bal_all=pd.concat(frames)
dat_bal=dat_bal_all[(dat_bal_all['odd']<=4)]     ## 只要“还款中”和“逾期”的合同
dat_bal['contract_id']=dat_bal['CONTRACTNO']
dat_bal.shape

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(897876, 7)

In [4]:
dat_bal.columns

Index(['data_date', 'CONTRACTNO', 'odd', 'cts', 'bal', 'bal_corpus',
       'contract_id'],
      dtype='object')

## 导入hue平台内的客户信息表

In [5]:
hue_to1805=pd.read_csv('E:\\pydata\\jhjj_PQR\\hue_1712_1805.csv',encoding='gbk')
hue_to1807=pd.read_csv('E:\\pydata\\jhjj_PQR\\hue_1806_1807.csv',encoding='gbk')
hue_to1809=pd.read_csv('E:\\pydata\\jhjj_PQR\\hue_1808_1809.csv',encoding='gbk')
hue_to1811=pd.read_csv('E:\\pydata\\jhjj_PQR\\hue_1810_1811.csv',encoding='gbk')

In [6]:
frames=[hue_to1805,hue_to1807,hue_to1809,hue_to1811]
dat_hue_all=pd.concat(frames)
dat_hue=dat_hue_all

In [7]:
dat_hue.head()

,contract_id,apply_type,regist_utm_source,apply_utm_source,age,sex,score,model_id,apply_time,month_apply,day_apply
0,1.000478e+10,白名单,wechat_public,h5,24.0,男,909.0,M_MIL_XGB_20170915,2017-12-13 00:01:43,2017-12,2017-12-13
1,1.000478e+10,白名单,信用管家,h5,34.0,男,980.0,M_MIL_XGB_20170915,2017-12-13 00:20:17,2017-12,2017-12-13
2,1.000478e+10,白名单,融之家,h5,49.0,男,976.0,M_MIL_XGB_20170915,2017-12-13 00:27:19,2017-12,2017-12-13
3,1.000478e+10,再贷,web,h5,28.0,男,0.0,M_MIL_XGB_20170915,2017-12-13 00:32:14,2017-12,2017-12-13
4,1.000478e+10,白名单,融之家_联合,OA_86,54.0,男,978.0,M_MIL_XGB_20170915,2017-12-13 00:40:21,2017-12,2017-12-13


## merge匹配余额的特征信息

In [8]:
dat=pd.merge(dat_bal,dat_hue,how='left')
dat_newMil=dat[dat['month_apply']>= '2017-12']

In [9]:
# 数据框操作
dat_newMil['score_cut']=dat_newMil['score'].apply(lambda x: '900-1000' if x >=900 
                          else '800-900' if x >=800 
                          else '700-800' if x >=700
                          else '600-700' if x >=600
                          else '<600')
dat_newMil['age_cut']=dat_newMil['age'].apply(lambda x: '-24' if x<=24 
                                              else '25-29' if x<=29
                                              else '30-34' if x<=34
                                              else '35-39' if x<=39
                                              else '40-44' if x<=44
                                              else '45-59' if x<=59
                                              else '60+')
dat_newMil['channel_reg']=dat_newMil['regist_utm_source'].apply(lambda x: x if x in ['wechat_browser','android','融之家','借了吗','现金白卡','信用无忧','信用管家']
                                                                else '现金白卡' if x=='现金白卡2' 
                                                                else '其他渠道')
                                                                                                                                

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

In [10]:
dat_newMil.head()

,data_date,CONTRACTNO,odd,cts,bal,bal_corpus,contract_id,apply_type,regist_utm_source,apply_utm_source,age,sex,score,model_id,apply_time,month_apply,day_apply,score_cut,age_cut,channel_reg
59106,2018/1/31,10004781704,0,1,1720.0,1600.0,10004781704,白名单,wechat_public,h5,24.0,男,909.0,M_MIL_XGB_20170915,2017-12-13 00:01:43,2017-12,2017-12-13,900-1000,-24,其他渠道
59107,2018/1/31,10004781749,0,1,1720.0,1600.0,10004781749,白名单,信用管家,h5,34.0,男,980.0,M_MIL_XGB_20170915,2017-12-13 00:20:17,2017-12,2017-12-13,900-1000,30-34,信用管家
59108,2018/1/31,10004781757,0,1,1720.0,1600.0,10004781757,白名单,融之家,h5,49.0,男,976.0,M_MIL_XGB_20170915,2017-12-13 00:27:19,2017-12,2017-12-13,900-1000,45-59,融之家
59109,2018/1/31,10004781780,0,1,1720.0,1600.0,10004781780,再贷,web,h5,28.0,男,0.0,M_MIL_XGB_20170915,2017-12-13 00:32:14,2017-12,2017-12-13,<600,25-29,其他渠道
59110,2018/1/31,10004781792,0,1,1720.0,1600.0,10004781792,白名单,融之家_联合,OA_86,54.0,男,978.0,M_MIL_XGB_20170915,2017-12-13 00:40:21,2017-12,2017-12-13,900-1000,45-59,其他渠道


In [68]:
#dat_newMil[(dat_newMil['data_date']=='2018/11/30') & (dat_newMil['odd']<=3)]['bal_corpus'].sum()

In [67]:
#dat_newMil[(dat_newMil['odd']<=3)&(dat_newMil['cts']==9)]

## 聚合瘦身

In [63]:
# 预处理 解决数据类型问题及缺失值问题
dat_newMil.dtypes
dat_newMil['odd']=dat_newMil['odd'].astype(np.object)
dat_newMil['cts']=dat_newMil['cts'].astype(np.object)
dat_newMil['model_id']=dat_newMil['model_id'].fillna('null')
#dat_newMil.dtypes

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [64]:
# 聚合
groupby_key=['data_date','odd','cts','sex','model_id','month_apply','score_cut','age_cut','channel_reg']
dat_grp=dat_newMil.groupby(by=groupby_key,as_index=False).agg({'bal_corpus':sum})

In [14]:
dat_newMil.columns

Index(['data_date', 'CONTRACTNO', 'odd', 'cts', 'bal', 'bal_corpus',
       'contract_id', 'apply_type', 'regist_utm_source', 'apply_utm_source',
       'age', 'sex', 'score', 'model_id', 'apply_time', 'month_apply',
       'day_apply', 'score_cut', 'age_cut', 'channel_reg'],
      dtype='object')

In [15]:
dat_grp.head()

,data_date,odd,cts,regist_utm_source,apply_utm_source,sex,model_id,month_apply,score_cut,age_cut,channel_reg,bal_corpus
0,2018/1/31,0,1,99贷,OA_109,男,M_MIL_XGB_TD_20171027,2018-01,800-900,40-44,其他渠道,2000.0
1,2018/1/31,0,1,99贷,OA_116,女,M_MIL_XGB_TD_20171027,2018-01,800-900,30-34,其他渠道,2000.0
2,2018/1/31,0,1,99贷,h5,女,M_MIL_XGB_20170915,2017-12,900-1000,25-29,其他渠道,6400.0
3,2018/1/31,0,1,99贷,h5,女,M_MIL_XGB_20170915,2017-12,900-1000,30-34,其他渠道,3200.0
4,2018/1/31,0,1,99贷,h5,女,M_MIL_XGB_20170915,2017-12,900-1000,40-44,其他渠道,1600.0


## 导出至CSV

In [ ]:
## 导出明细CSV
dat_newMil.to_csv('E:\\pydata\\jhjj_PQR\\jhjj_pqr.csv',encoding='gbk')

In [65]:
## 导出聚合CSV
dat_grp.to_csv('E:\\pydata\\jhjj_PQR\\jhjj_pqr_grp.csv',encoding='gbk')